## Test avec différents paramétres : 




In [4]:
# pip install pandas


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.5 MB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 5.2 MB/s  eta 0:00:01
     |████████████████████████████████| 773 kB 35.3 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 4.8 MB/s  eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Lecture des données :

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import common as cmn
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, roc_auc_score, accuracy_score


from scipy.ndimage import gaussian_filter


import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.model_selection._validation")

In [3]:
fname = "./datasets/AFDpresidentutf8/corpus.tache1.learn.utf8"
alltxts,alllabs = cmn.load_pres(fname)


## Fonction grid_search

In [4]:

def grid_search (parameters , scoring,preprocessors ,score_to_maximize):
    pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(preprocessor=preprocessors)),
    ('reg', LogisticRegression())
    ])

    [X_all_train, X_all_test, Y_train, y_test]  = train_test_split(alltxts, alllabs, test_size=0.3, random_state=10, shuffle=True)
    
    
    grid_search = GridSearchCV(pipeline, parameters,scoring=scoring, refit=score_to_maximize, cv=5, n_jobs=-1, verbose=1)
    
    
    grid_search.fit(X_all_train, Y_train)
    
    
    print("Meilleurs paramètres trouvés:")
    print(grid_search.best_params_)
    #_____________
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_all_test)
    
    smoothed_pred = gaussian_filter(y_pred,sigma=0.1)
    
    # Calcul des métriques de performance après le lissage

    f1 = f1_score(y_test, smoothed_pred,  pos_label=-1)  # or 'macro' or 'weighted'

    roc_auc = roc_auc_score(y_test, smoothed_pred)
    accuracy = accuracy_score(y_test, (smoothed_pred > 0.5).astype(int))
    
    print("Performance après lissage:")
    print("F1 Score:", f1)
    print("AUC:", roc_auc)
    print("Accuracy:", accuracy)
    
    
    
    
    #_____________
    # print("Scores:")
    # print("F1 Score:", grid_search.cv_results_['mean_test_f1_score'])
    # print("AUC:", grid_search.cv_results_['mean_test_roc_auc'])
    # print("Accuracy:", grid_search.cv_results_['mean_test_accuracy'])


## Test 1 :
On essaye de penaliser notre classe minoritaire avec diffèrentes vaaleurs de pénaisation 

In [35]:
class_weights = {
    1: 1.0,  # Class 1, the majority class, gets weight 1.0 (default weight)
    -1: 5.0  # Class -1, the minority class, gets weight 5.0
}


parameters = {
    'tfidf__max_df': [ 0.2,0.5],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [100000],
    'reg__class_weight': [{1: 1, -1: w} for w in [1, 5, 10, 20]],  
    'reg__max_iter': [10000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 1,10,100]
}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: ((cmn.suppression_chiffres(cmn.majuscules_en_marqueurs((cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 100000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.9050743149094286
AUC: 0.7827068919817048
Accuracy: 0.8226892707849512


### Test 2 :
Maximiser le f1 score de la classe minoritaire

In [30]:
class_weights = {
    1: 1.0,  # Class 1, the majority class, gets weight 1.0 (default weight)
    -1: 5.0  # Class -1, the minority class, gets weight 5.0
}


parameters = {
    'tfidf__max_df': [ 0.5],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [40000,100000],
    'reg__class_weight': ['balanced',class_weights],  
    'reg__max_iter': [10000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 1,10,100]
}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: (((cmn.majuscules_en_marqueurs((cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__class_weight': {1: 1.0, -1: 5.0}, 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 100000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.9040320473743795
AUC: 0.782878231512926
Accuracy: 0.8230427588609248


### Test 3 :
Maximiser roc auc score

In [28]:
class_weights = {
    1: 1.0,  # Class 1, the majority class, gets weight 1.0 (default weight)
    -1: 5.0  # Class -1, the minority class, gets weight 5.0
}


parameters = {
    'tfidf__max_df': [ 0.5],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [40000,100000],
    'reg__class_weight': ['balanced',class_weights],  
    'reg__max_iter': [10000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 1,10,100]
}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: (((cmn.majuscules_en_marqueurs((cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'roc_auc')


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Meilleurs paramètres trouvés:
{'reg__C': 1, 'reg__class_weight': 'balanced', 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 40000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.8680658364538884
AUC: 0.8090931030390787
Accuracy: 0.7726203953670644


### Test 4:
essayer de maximiser des deux

In [35]:
class_weights = {
    1: 1.0,  # Class 1, the majority class, gets weight 1.0 (default weight)
    -1: 5.0  # Class -1, the minority class, gets weight 5.0
}


parameters = {
    'tfidf__max_df': [ 0.2,0.5],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [100000],
    'reg__class_weight': [{1: 1, -1: w} for w in [1, 5, 10, 20]],  
    'reg__max_iter': [10000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 1,10,100]
}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: ((cmn.suppression_chiffres(cmn.majuscules_en_marqueurs((cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 100000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.9050743149094286
AUC: 0.7827068919817048
Accuracy: 0.8226892707849512


## POur tester sur mon pc :

In [4]:
class_weights = {
    1: 1.0,  # Class 1, the majority class, gets weight 1.0 (default weight)
    -1: 5.0  # Class -1, the minority class, gets weight 5.0
}


parameters = {
    'tfidf__max_df': [ 0.2,0.5],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [100000],
    'reg__class_weight': [{1: 1, -1: w} for w in [1, 5, 10, 20]],  
    'reg__max_iter': [10000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 1,10,100]
}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: ((cmn.suppression_chiffres(cmn.majuscules_en_marqueurs((cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 100000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.9050743149094286
AUC: 0.7827068919817048
Accuracy: 0.8226892707849512


In [8]:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 100000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
class_weights = {
    1: 1.0,  # Class 1, the majority class, gets weight 1.0 (default weight)
    -1: 5.0  # Class -1, the minority class, gets weight 5.0
}


parameters = {
    'tfidf__max_df': [ 0.5],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [100000],
    'reg__class_weight': [{1: 1, -1: 5} ],  
    'reg__max_iter': [10000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 10]
}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: ((cmn.suppression_chiffres(cmn.majuscules_en_marqueurs(cmn.suppression_ponctuation(cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 100000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.6344735077129444
AUC: 0.7827068919817048
Accuracy: 0.8226892707849512


In [26]:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 100000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
class_weights = {
    1: 1.0,  # Class 1, the majority class, gets weight 1.0 (default weight)
    -1: 5.0  # Class -1, the minority class, gets weight 5.0
}


parameters = {
    'tfidf__max_df': [ 0.45],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [90000],
    'reg__class_weight': [{1: 1, -1: 5} ],  
    'reg__max_iter': [10000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 10]
}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: ((cmn.suppression_chiffres(cmn.majuscules_en_marqueurs(cmn.suppression_ponctuation(cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.45, 'tfidf__max_features': 90000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.6349419124218052
AUC: 0.7831072171504867
Accuracy: 0.82263121226196


In [29]:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 100000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
class_weights = {
    1: 1.0,  # Class 1, the majority class, gets weight 1.0 (default weight)
    -1: 5.0  # Class -1, the minority class, gets weight 5.0
}


parameters = {
    'tfidf__max_df': [ 0.45],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [90000],
    'reg__class_weight': [{1: 1, -1: 5} ],  
    'reg__max_iter': [1000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 10]
}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: ((((cmn.suppression_ponctuation(cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 1000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.45, 'tfidf__max_features': 90000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.6361607142857143
AUC: 0.7839748961092721
Accuracy: 0.82263121226196


In [30]:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 100000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
class_weights = {
    1: 1.0,  # Class 1, the majority class, gets weight 1.0 (default weight)
    -1: 5.0  # Class -1, the minority class, gets weight 5.0
}


parameters = {
    'tfidf__max_df': [ 0.45],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [90000],
    'reg__class_weight': [{1: 1, -1: 5} ],  
    'reg__max_iter': [1000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 10]
}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: ((cmn.majuscules_en_marqueurs((cmn.suppression_ponctuation(cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 1000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.45, 'tfidf__max_features': 90000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.6361607142857143
AUC: 0.7839748961092721
Accuracy: 0.82263121226196


In [37]:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 10000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 100000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
class_weights = {
    1: 1.0,  # Class 1, the majority class, gets weight 1.0 (default weight)
    -1: 5.0  # Class -1, the minority class, gets weight 5.0
}


parameters = {
    'tfidf__max_df': [ 0.45],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [90000],
    'reg__class_weight': [{1: 1, -1: 5} ],  
    'reg__max_iter': [1000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 10]
}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: (cmn.lemmatization(cmn.majuscules_en_marqueurs((cmn.suppression_ponctuation(cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 1000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.45, 'tfidf__max_features': 90000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.6296296296296295
AUC: 0.7804017907837535
Accuracy: 0.8217022758941013


### Stop words : 

In [55]:
preprocessors = lambda x: ((cmn.majuscules_en_marqueurs((cmn.suppression_ponctuation(cmn.suppression_balises_html((x)))))))

vectorizer_bigram = CountVectorizer(ngram_range=(2, 2),preprocessor=preprocessors)
corpus_dicours = alltxts
corpus_dicours_sparse_mat_bigram = vectorizer_bigram.fit_transform(corpus_dicours) # Output is a sparse matrix
print("Taille initiale du vocabulaire avec des bigrammes :",len(vectorizer_bigram.get_feature_names_out()))
frequence = np.array(corpus_dicours_sparse_mat_bigram.sum(axis=0))[0]
indices_tries = np.argsort(-frequence, kind='quicksort')

# Trier les sommes des colonnes en utilisant les indices triés
somme_colonnes_triees = [frequence[i] for i in indices_tries]
somme_colonnes_triees

bigrammes = vectorizer_bigram.get_feature_names_out()
bigrammes_100_plus_frequents = [bigrammes[i] for i in indices_tries[:100] ]
list(bigrammes_100_plus_frequents)

Taille initiale du vocabulaire avec des bigrammes : 325503


['de la',
 'et de',
 'la France',
 'aujourd hui',
 'et la',
 'dans le',
 'de notre',
 'qu il',
 'de nos',
 'que nous',
 'tous les',
 'et les',
 'dans la',
 'Monsieur le',
 'que la',
 'que vous',
 'dans les',
 'et des',
 'qui est',
 'que les',
 'et le',
 'vous avez',
 'est pas',
 'et qui',
 'pour la',
 'le Président',
 'que je',
 'qui ont',
 'ceux qui',
 'est un',
 'sur le',
 'ce que',
 'ce qui',
 'de ce',
 'de cette',
 'il faut',
 'de votre',
 'du monde',
 'qu elle',
 'le monde',
 'nous avons',
 'que le',
 'sur la',
 'est une',
 'par la',
 'de ses',
 'est la',
 'pour les',
 'les plus',
 'Il faut',
 'et je',
 'qu ils',
 'est le',
 'doit être',
 'la paix',
 'toutes les',
 'la vie',
 'notre pays',
 'de ces',
 'par le',
 'il est',
 'de son',
 'La France',
 'sur les',
 'de leur',
 'de tous',
 'qui sont',
 'mais aussi',
 'la République',
 'plus de',
 'Union européenne',
 'qui se',
 'est aussi',
 'par les',
 'entre les',
 'nous devons',
 'dans un',
 'de Europe',
 'pour le',
 'que on',
 'avec 

In [14]:

vectorizer_bigram = CountVectorizer(ngram_range=(1, 2),preprocessor=preprocessors)
corpus_dicours = alltxts
corpus_dicours_sparse_mat_bigram = vectorizer_bigram.fit_transform(corpus_dicours) # Output is a sparse matrix
print("Taille initiale du vocabulaire avec des bigrammes :",len(vectorizer_bigram.get_feature_names_out()))
frequence = np.array(corpus_dicours_sparse_mat_bigram.sum(axis=0))[0]
indices_tries = np.argsort(-frequence, kind='quicksort')

# Trier les sommes des colonnes en utilisant les indices triés
somme_colonnes_triees = [frequence[i] for i in indices_tries]
somme_colonnes_triees

bigrammes = vectorizer_bigram.get_feature_names_out()
unigrammes_100_plus_frequents_12 = [bigrammes[i] for i in indices_tries[:100] ]
print((unigrammes_100_plus_frequents_12))
print((somme_colonnes_triees[:100]))

Taille initiale du vocabulaire avec des bigrammes : 356716
['de', 'la', 'et', 'le', 'les', 'des', 'est', 'que', 'qui', 'en', 'un', 'une', 'de la', 'pour', 'dans', 'du', 'vous', 'nous', 'au', 'plus', 'ce', 'il', 'pas', 'qu', 'je', 'par', 'notre', 'France', 'ne', 'sur', 'nos', 'avec', 'se', 'pays', 'sont', 'et de', 'aussi', 'Je', 'aux', 'cette', 'ont', 'être', 'leur', 'Il', 'tout', 'la France', 'tous', 'votre', 'son', 'La', 'même', 'elle', 'ou', 'ses', 'bien', 'entre', 'Europe', 'mais', 'hui', 'comme', 'on', 'ces', 'sa', 'doit', 'monde', 'faire', 'ai', 'aujourd', 'aujourd hui', 'et la', 'dans le', 'Et', 'faut', 'été', 'sans', 'fait', 'de notre', 'qu il', 'Mais', 'si', 'Nous', 'Le', 'où', 'leurs', 'avez', 'de nos', 'dire', 'Les', 'ils', 'très', 'deux', 'dont', 'peut', 'développement', 'que nous', 'tous les', 'et les', 'ensemble', 'dans la', 'Elle']
[68623, 39750, 35180, 24366, 24362, 21495, 17247, 16081, 15568, 15473, 12257, 12012, 11919, 11033, 10962, 10639, 8131, 7735, 7571, 7266, 6585,

In [78]:


parameters = {
    # 'tfidf__max_df': [ 0.45],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [90000],
    'tfidf__stop_words':[bigrammes_100_plus_frequents[:10]+unigrammes_100_plus_frequents[:5]] ,
    'reg__class_weight': [{1: 1, -1: 5} ],  
    'reg__max_iter': [1000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 10]

}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: ((cmn.majuscules_en_marqueurs((cmn.suppression_ponctuation(cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/home/djeghali/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['France', 'aujourd', 'dans', 'hui', 'il', 'nos', 'notre', 'nous', 'qu', 'que'] not in stop_words.
  warnings.warn(


Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 1000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_features': 90000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2), 'tfidf__stop_words': ['de la', 'et de', 'la France', 'aujourd hui', 'et la', 'dans le', 'de notre', 'qu il', 'de nos', 'que nous', 'de', 'la', 'et', 'le', 'les']}
Performance après lissage:
F1 Score: 0.6383259911894273
AUC: 0.7879744546800008
Accuracy: 0.8205411054342777


In [81]:


parameters = {
    # 'tfidf__max_df': [ 0.45],
    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [90000],
    # 'tfidf__stop_words':[unigrammes_100_plus_frequents_12[:5]] ,
    'reg__class_weight': [{1: 1, -1: 5} ],  
    'reg__max_iter': [1000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 10]

}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: ((cmn.majuscules_en_marqueurs((cmn.suppression_ponctuation(cmn.suppression_balises_html((x)))))))

grid_search (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__class_weight': {1: 1, -1: 5}, 'reg__max_iter': 1000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_features': 90000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.6348214285714285
AUC: 0.7832235939583514
Accuracy: 0.8224570366929865


### test ppti : avec over sample


In [10]:
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from scipy.ndimage import gaussian_filter
from imblearn.over_sampling import SMOTE

def grid_search_ov(parameters, scoring, preprocessors, score_to_maximize):
    # Diviser les données en ensembles d'entraînement et de test
    X_all_train, X_all_test, Y_train, y_test = train_test_split(alltxts, alllabs, test_size=0.3, random_state=10, shuffle=True)
    
    # Créer une instance de TfidfVectorizer avec les préprocesseurs spécifiés
    tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessors)
    
    # Créer un pipeline avec TfidfVectorizer, suréchantillonnage SMOTE et LogisticRegression
    pipeline = Pipeline([
        ('tfidf', tfidf_vectorizer),
        ('smote', SMOTE()),  # Suréchantillonnage SMOTE
        ('reg', LogisticRegression())
    ])
    
    # Initialiser la grille de recherche
    grid_search = GridSearchCV(pipeline, parameters, scoring=scoring, refit=score_to_maximize, cv=5, n_jobs=-1, verbose=1)
    
    # Effectuer la recherche sur grille
    grid_search.fit(X_all_train, Y_train)
    
    # Afficher les meilleurs paramètres trouvés
    print("Meilleurs paramètres trouvés:")
    print(grid_search.best_params_)
    
    # Obtenir le meilleur modèle
    best_model = grid_search.best_estimator_
    
    # Prédire les étiquettes sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_all_test)
    
    # Lissage des prédictions
    smoothed_pred = gaussian_filter(y_pred, sigma=0.1)
    
    # Calculer les métriques de performance après lissage
    f1 = f1_score(y_test, smoothed_pred, pos_label=-1)
    roc_auc = roc_auc_score(y_test, smoothed_pred)
    accuracy = accuracy_score(y_test, (smoothed_pred > 0.5).astype(int))
    
    print("Performance après lissage:")
    print("F1 Score:", f1)
    print("AUC:", roc_auc)
    print("Accuracy:", accuracy)


In [27]:

parameters = {
    'tfidf__max_df': [ 0.45],

    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [90000],
    'reg__max_iter': [1000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 10]

}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: (cmn.suppression_chiffres(cmn.majuscules_en_marqueurs((cmn.suppression_ponctuation(cmn.suppression_balises_html((x)))))))

grid_search_ov (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 1 candidates, totalling 5 fits


Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__max_iter': 1000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.45, 'tfidf__max_features': 90000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.5801011804384486
AUC: 0.7628558496430253
Accuracy: 0.8044588945657223


In [30]:

parameters = {
    'tfidf__max_df': [ 0.5],

    'tfidf__min_df': [2],
    'tfidf__ngram_range': [ (1, 2)],
    'tfidf__binary': [True],
    'tfidf__max_features': [90000],
    'reg__max_iter': [1000],
    'reg__tol': [1e-4],
    'reg__penalty': [ 'l2' ],
    'reg__C': [ 10]

}

scoring = {
    'f1_score': make_scorer(f1_score,pos_label=-1),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}
preprocessors = lambda x: (cmn.suppression_chiffres(cmn.majuscules_en_marqueurs((cmn.suppression_ponctuation(cmn.suppression_balises_html((x)))))))

grid_search_ov (parameters , scoring,preprocessors ,'f1_score')


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Meilleurs paramètres trouvés:
{'reg__C': 10, 'reg__max_iter': 1000, 'reg__penalty': 'l2', 'reg__tol': 0.0001, 'tfidf__binary': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': 90000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Performance après lissage:
F1 Score: 0.5804311774461028
AUC: 0.7661851220015414
Accuracy: 0.8012076172782164
